In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.path import Path
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set default tensor type to float32
torch.set_default_tensor_type(torch.FloatTensor)

# Define the neural network architecture
class Net(nn.Module):
       def __init__(self):
           super(Net, self).__init__()
           self.hidden1 = nn.Linear(2, 64)
           self.hidden2 = nn.Linear(64, 128)
           self.hidden3 = nn.Linear(128, 256)
           self.hidden4 = nn.Linear(256, 128)
           self.hidden5 = nn.Linear(128, 64)
           self.output = nn.Linear(64, 1)

       def forward(self, x):
           x = torch.tanh(self.hidden1(x))
           x = torch.tanh(self.hidden2(x))
           x = torch.tanh(self.hidden3(x))
           x = torch.tanh(self.hidden4(x))
           x = torch.tanh(self.hidden5(x))
           return self.output(x)
    # Define the problem domain using the given vertices
vertices = np.array([
    [0, 0],
    [0, 3],
    [3, 3],
    [5, 2],
    [7, 2],
    [7,0],
    [0,0]  # Closing the polygon
],dtype=np.float32)
path = Path(vertices)
def in_domain(x, y):
    points = np.column_stack((x.cpu().numpy(), y.cpu().numpy()))
    return torch.tensor(path.contains_points(points), dtype=torch.bool, device=device)

# Define boundary conditions
def BC_bottom(x, y):
    return ((y == 0) & (x >= 0) & (x <= 7)).squeeze()

def BC_left(x, y):
    return ((x == 0) & (y >= 0) & (y <= 3)).squeeze()

def BC_top(x, y):
    return (((y == 3) & (x >= 0) & (x <= 3)) | 
            ((y - 2) / (-1) == (x - 3) / 2) & (x > 3) & (x < 5) |
            (y == 2) & (x >= 5) & (x <= 7)).squeeze()

def BC_right(x, y):
    return ((x == 7) & (y >= 0) & (y <= 2)).squeeze()

def BC(xy, net_u, net_v):
    x, y = xy[:, 0].unsqueeze(1), xy[:, 1].unsqueeze(1)
    
    u = net_u(xy)
    v = net_v(xy)
    
    bc_b = BC_bottom(x, y)
    bc_l = BC_left(x, y)
    bc_t = BC_top(x, y)
    bc_r = BC_right(x, y)
    
    loss = torch.mean(u[bc_b]**2 + v[bc_b]**2)  # ux = uy = 0 on bottom
    loss += torch.mean(u[bc_l]**2)  # ux = 0 on left side
    loss += torch.mean(u[bc_r]**2)  # uy = 0 on right side
    
    # Stress-free condition at the top (σyy = 0, σxy = 0)
    xy_top = xy[bc_t].requires_grad_(True)
    u_top = net_u(xy_top)
    v_top = net_v(xy_top)
    
    u_x_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 0]
    u_y_top = torch.autograd.grad(u_top.sum(), xy_top, create_graph=True)[0][:, 1]
    v_x_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 0]
    v_y_top = torch.autograd.grad(v_top.sum(), xy_top, create_graph=True)[0][:, 1]
    
    E = 5.0  # Young's modulus
    nu = 0.3  # Poisson's ratio
    
    sigma_yy_top = E / (1 - nu**2) * (v_y_top + nu * u_x_top)
    sigma_xy_top = E / (2 * (1 + nu)) * (u_y_top + v_x_top)
    
    loss += torch.mean(sigma_yy_top**2 + sigma_xy_top**2)
    
    return loss

def load_fem_data(filename='FEM2_data.csv'):
    df = pd.read_csv(filename)
    x = torch.tensor(df['X'].values, dtype=torch.float32).unsqueeze(1).to(device)
    y = torch.tensor(df['Y'].values, dtype=torch.float32).unsqueeze(1).to(device)
    return x, y

# Generate training data
def generate_training_data(fem_data_file, n_boundary):
    x, y = load_fem_data(fem_data_file)
    
    # Keep only points inside the domain
    mask = in_domain(x, y)
    x, y = x[mask], y[mask]
    
    # Generate boundary points
    t = torch.linspace(0, 1, n_boundary, device=device).unsqueeze(1)
    
    # Define the boundary segments
    segments = [
        ([0, 0, 0], [0, 3, 3]),  # Left boundary
        ([0, 3, 5, 7, 7], [3, 3, 2, 2, 0]),  # Top and right boundary
        ([7, 0], [0, 0])  # Bottom boundary
    ]
    
    x_b = []
    y_b = []
    
    for segment in segments:
        x_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[0])), segment[0]), dtype=torch.float32, device=device)
        y_seg = torch.tensor(np.interp(t.cpu().numpy(), np.linspace(0, 1, len(segment[1])), segment[1]), dtype=torch.float32, device=device)
        x_b.append(x_seg)
        y_b.append(y_seg)
    
    x_b = torch.cat(x_b)
    y_b = torch.cat(y_b)
    
    return x, y, x_b, y_b

def PDE(x, y, net_u, net_v):
    xy = torch.cat([x, y], dim=1)
    xy.requires_grad = True
    
    u = net_u(xy)
    v = net_v(xy)
    
    u_x = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    u_y = torch.autograd.grad(u.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    v_x = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1)
    v_y = torch.autograd.grad(v.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1)
    
    E = 5  # Young's modulus
    nu = 0.3  # Poisson's ratio
    gamma = 1
    sigma_xx = E / (1 - nu**2) * (u_x + nu * v_y)
    sigma_yy = E / (1 - nu**2) * (v_y + nu * u_x)
    sigma_xy = E / (2 * (1 + nu)) * (u_y + v_x)
    
    f_x = torch.zeros_like(x)
    f_y = -gamma * torch.ones_like(y)  # Body force
    
    R_x = torch.autograd.grad(sigma_xx.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_x
    R_y = torch.autograd.grad(sigma_xy.sum(), xy, create_graph=True)[0][:, 0].unsqueeze(1) + \
          torch.autograd.grad(sigma_yy.sum(), xy, create_graph=True)[0][:, 1].unsqueeze(1) + f_y
    
    loss_x = torch.mean(R_x**2)
    loss_y = torch.mean(R_y**2)
    
    return loss_x, loss_y


def train(net_u, net_v, optimizer, n_epochs, fem_data_file):
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=500, verbose=True)
    
    best_loss = float('inf')
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        x, y, x_b, y_b = generate_training_data(fem_data_file, 30000)
        xy = torch.cat([x, y], dim=1)
        xy_b = torch.cat([x_b, y_b], dim=1)
        
        loss_pde_x, loss_pde_y = PDE(x, y, net_u, net_v)
        loss_bc = BC(xy_b, net_u, net_v)
        
        loss = loss_pde_x + 2 * loss_pde_y + 10 * loss_bc
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(list(net_u.parameters()) + list(net_v.parameters()), max_norm=0.5)
        optimizer.step()
        
        # Step the scheduler
        scheduler.step(loss)
        
        if loss < best_loss:
            best_loss = loss
            torch.save({
                'net_u_state_dict': net_u.state_dict(),
                'net_v_state_dict': net_v.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, 'best_model.pth')
        
        if (epoch + 1) % 100 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.6f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')

def plot_results(net_u, net_v, fem_data_file='FEM2_data.csv', output_filename='PiNN2_data.csv'):
    # Load FEM data
    df_fem = pd.read_csv(fem_data_file)
    X = df_fem['X'].values
    Y = df_fem['Y'].values
    
    # Create tensor from FEM data
    XY = torch.tensor(np.column_stack([X, Y]), dtype=torch.float32).to(device)
    
    # Compute displacements and stresses for all points
    XY.requires_grad_(True)
    with torch.enable_grad():
        U = net_u(XY)
        V = net_v(XY)
        
        U_x = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 0]
        U_y = torch.autograd.grad(U.sum(), XY, create_graph=True)[0][:, 1]
        V_x = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 0]
        V_y = torch.autograd.grad(V.sum(), XY, create_graph=True)[0][:, 1]
        
        E = 5  # Young's modulus
        nu = 0.3  # Poisson's ratio
        
        sigma_xx = E / (1 - nu**2) * (U_x + nu * V_y)
        sigma_yy = E / (1 - nu**2) * (V_y + nu * U_x)
        sigma_xy = E / (2 * (1 + nu)) * (U_y + V_x)
    
    # Move tensors to CPU and convert to numpy
    U_full = U.detach().cpu().numpy().squeeze()
    V_full = V.detach().cpu().numpy().squeeze()
    sigma_xx_full = sigma_xx.detach().cpu().numpy().squeeze()
    sigma_yy_full = sigma_yy.detach().cpu().numpy().squeeze()
    sigma_xy_full = sigma_xy.detach().cpu().numpy().squeeze()
    
    # Calculate displacement magnitude
    magnitude = np.sqrt(U_full**2 + V_full**2)
    
    # Create DataFrame for CSV export
    df_out = pd.DataFrame({
        'X': X,
        'Y': Y,
        'ux': U_full,
        'uy': V_full,
        'sigma_xx': sigma_xx_full,
        'sigma_yy': sigma_yy_full,
        'sigma_xy': sigma_xy_full,
        'magnitude': magnitude
    })

    # Save the output to a CSV file
    df_out.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")
    
    # Apply domain mask for plotting
    mask = in_domain(XY[:, 0], XY[:, 1])
    mask_cpu = mask.cpu().numpy()
    
    # Create masked arrays for plotting
    U_masked = np.ma.masked_array(U_full, mask=~mask_cpu)
    V_masked = np.ma.masked_array(V_full, mask=~mask_cpu)
    sigma_xx_masked = np.ma.masked_array(sigma_xx_full, mask=~mask_cpu)
    sigma_yy_masked = np.ma.masked_array(sigma_yy_full, mask=~mask_cpu)
    sigma_xy_masked = np.ma.masked_array(sigma_xy_full, mask=~mask_cpu)
    magnitude_masked = np.ma.masked_array(magnitude, mask=~mask_cpu)
    
    # Plot the results
    plt.figure(figsize=(20, 15))
    
    # Plot u displacement (ux)
    plt.subplot(231)
    sc = plt.scatter(X, Y, c=U_masked, cmap='jet')
    plt.colorbar(sc, label='ux displacement')
    plt.title('ux displacement')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Plot v displacement (uy)
    plt.subplot(232)
    sc = plt.scatter(X, Y, c=V_masked, cmap='jet')
    plt.colorbar(sc, label='uy displacement')
    plt.title('uy displacement')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Plot sigma_xx
    plt.subplot(233)
    sc = plt.scatter(X, Y, c=sigma_xx_masked, cmap='jet')
    plt.colorbar(sc, label='sigma_xx')
    plt.title('sigma_xx')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Plot sigma_yy
    plt.subplot(234)
    sc = plt.scatter(X, Y, c=sigma_yy_masked, cmap='jet')
    plt.colorbar(sc, label='sigma_yy')
    plt.title('sigma_yy')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Plot sigma_xy
    plt.subplot(235)
    sc = plt.scatter(X, Y, c=sigma_xy_masked, cmap='jet')
    plt.colorbar(sc, label='sigma_xy')
    plt.title('sigma_xy')
    plt.xlabel('X')
    plt.ylabel('Y')
    
    # Plot displacement magnitude
    plt.subplot(236)
    sc = plt.scatter(X, Y, c=magnitude_masked, cmap='jet')
    plt.colorbar(sc, label='Displacement magnitude')
    plt.title('Displacement magnitude')
    plt.xlabel('X')
    plt.ylabel('Y')

    plt.tight_layout()
    plt.savefig('PiNN_results.png')
    plt.show()

# Main execution
if __name__ == "__main__":
    net_u = Net().to(device)
    net_v = Net().to(device)
    
    optimizer = optim.Adam(list(net_u.parameters()) + list(net_v.parameters()), lr=0.001)
    
    fem_data_file = 'FEM2_data.csv'
    train(net_u, net_v, optimizer, n_epochs=1000, fem_data_file=fem_data_file)
    
    # Load the best model
    checkpoint = torch.load('best_model.pth')
    net_u.load_state_dict(checkpoint['net_u_state_dict'])
    net_v.load_state_dict(checkpoint['net_v_state_dict'])
    
    try:
        plot_results(net_u, net_v, fem_data_file=fem_data_file)
    except Exception as e:
        print(f"Error in plot_results: {str(e)}")

Using device: cuda


c:\Users\apisi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)
c:\Users\apisi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 100/1000, Loss: 0.221831, LR: 0.001000
Epoch 200/1000, Loss: 0.149411, LR: 0.001000
Epoch 300/1000, Loss: 0.040693, LR: 0.001000
Epoch 400/1000, Loss: 0.031763, LR: 0.001000
Epoch 500/1000, Loss: 0.037954, LR: 0.001000
Epoch 600/1000, Loss: 0.033611, LR: 0.001000
Epoch 700/1000, Loss: 0.017168, LR: 0.001000


KeyboardInterrupt: 